In [1]:
import transformers
import torch
import os
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

/data/nlp/aditya/miniforge3/envs/lmharness/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Sentence = [
    "I hate Muslim terrorists and the psychotic Muslims in ISIS. Is that allowed in your politically correct version of reality?",
    "Apparently the emails state he buys meth from gay Islamic party boys who are transphobic hate blacks puppies and illegally voted for Hillary",
    "None of that would ever have happened if he werent black. What do you mean?",
    "1000000 women are just idiots? Your simpleminded generalization paints you as seeming to be one too"
]

label = [1, 1, 0, 0]

In [3]:
# define which model are you testing
model_name = "bert-large-uncased"

In [4]:
def load_model(model_name, task_name):
    # Load the configuration
    config_path = f"MTL-E/{task_name}_model/{model_name}/config.json"
    config = transformers.AutoConfig.from_pretrained(config_path)

    # Load the model state dict
    model_path = f"MTL-E/{task_name}_model/{model_name}/pytorch_model.bin"
    model = transformers.AutoModelForSequenceClassification.from_pretrained(
        pretrained_model_name_or_path=None,
        config=config,
        state_dict=torch.load(model_path)
    )

    # Load the tokenizer
    tokenizer = transformers.AutoTokenizer.from_pretrained(f"MTL-E/{task_name}_model/{model_name}")

    return model, tokenizer


In [5]:
# Function to load test data from CSV
def load_test_data(csv_path):
    data = pd.read_csv(csv_path)
    print(data)
    return data

In [6]:
def evaluate_model(model, tokenizer, test_data, task_name, batch_size=8):
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    model.to(device)
    true_list = []
    pred_list = []
    sentences = []
    
    for i in range(0, len(test_data), batch_size):
        batch_data = test_data.iloc[i:i+batch_size]
        inputs = tokenizer(batch_data['Sentence'].tolist(), max_length=128, padding=True, truncation=True, return_tensors="pt")
        labels = torch.tensor(batch_data['labels'].tolist())
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)
        
        with torch.no_grad():
            logits = model(**inputs).logits
        
        predictions = torch.argmax(logits, dim=1).cpu().numpy()
        true_list.extend(labels.cpu().numpy())
        pred_list.extend(predictions)
        sentences.extend(batch_data['Sentence'].tolist())
    
    # Calculate evaluation metrics
    conf_matrix = confusion_matrix(true_list, pred_list)
    class_report = classification_report(true_list, pred_list, output_dict=True)
    df_data = {
        "Sentence": sentences,
        "True": true_list,
        "Predicted": pred_list
    }
    df = pd.DataFrame(df_data)
    df.to_csv(f"{task_name}_results.csv", index=False)
    return conf_matrix, class_report

In [7]:
# Load models
bias_model, bias_tokenizer = load_model(model_name, "bias")
stereotype_model, stereotype_tokenizer = load_model(model_name, "stereotype")

# Evaluate models
for task_name, model, tokenizer in [("bias", bias_model, bias_tokenizer), ("stereotype", stereotype_model, stereotype_tokenizer)]:
    # Load test data
    #Give path to test data
    test_csv_path = "MTL-E/Dataset/StereoBias/stereotype/test.csv" if task_name=='stereotype' else "MTL-E/Dataset/ToxicBias/test.csv"
    test_data = load_test_data(test_csv_path)
    print(f"Evaluating {task_name} model...")
    conf_matrix, class_report = evaluate_model(model, tokenizer, test_data, task_name)

    print("---------------------------------Confusion Matrix------------------------------------")
    print(pd.DataFrame(conf_matrix))

    print("---------------------------------Evaluation Metrics------------------------------------")
    print(pd.DataFrame(class_report).transpose())
        
    # Save results to CSV
    result_csv_path = f"MTL-E/results/{task_name}_{model_name}_results.csv"
    pd.DataFrame(class_report).transpose().to_csv(result_csv_path, index=True)
    print(f"Results saved to {result_csv_path}")

/tmp/ipykernel_87208/3601068973.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=torch.load(model_path)
/tmp/ipykernel_87208/3601068973.py:11: FutureWarning: 

                                              Sentence  identity_attack_x  \
0     As someone pointed out earlier ISIS and radic...           0.800000   
1     URL bellamycharlottesvilletwitter  Read up on...           0.506024   
2    . . .  she wants to give other women of color ...           0.500000   
3    . The whole world hates the white world yet th...           0.500000   
4    ... what horrible human beings .... pathetic. ...           0.653333   
..                                                 ...                ...   
645  Youre right he should pick something else to b...           0.500000   
646  Youre right the international condemnation of ...           0.700000   
647  Youre the one making claims.  I called you on ...           0.600000   
648  Youre white so clearly you dont understand or ...           0.800000   
649  Yup I keep telling people people like you are ...           0.636364   

     labels          target       category  \
0         1  muslims,whites  

In [8]:
!pwd

/data/nlp/aditya/StereotypeAsCatalystForBias


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
